## Python Classes Strike Again

The Python implementation of classes has some peculiarities. The bare truth is that in Python the class of an object is an object itself. You can check this by issuing `type()` on the class

In [3]:
a = 1
print(type(a))

<class 'int'>


In [4]:
print(type(int))
# type(클래스명) == type 
# 클래스명의 type은 항상 type 이다.
# type(x) 결과가 type이면 x는 class이다.

# 선언은 2가지, 함수 , class
# 함수가 선언되면,  호출해야하고
# class가 선언되면, 인스턴스화 헤야한다.

<class 'type'>


This shows that the `int` class is an object, an instance of the `type` class.

This concept is not so difficult to grasp as it can seem at first sight: in the real world we deal with _concepts_ using them like _things_: for example we can talk about the concept of "door", telling people how a door looks like and how it works. In this case the concept of door is the topic of our discussion, so in our everyday experience the _type_ of an object is an object itself. In Python this can be expressed by saying that _everything is an object_.

If the class of an object is itself an instance it is a concrete object and is stored somewhere in memory. Let us leverage the _inspection_ capabilities of Python and its `id()` function to check the status of our objects. The `id()` built-in function returns the memory position of an object.

In the first post we defined this class

In [5]:
class Door:
    def __init__(self, number, status):
        self.number = number
        self.status = status
        
    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'
    
    # python에서는 인자 안보고 이름만 봄 
    # == 파이썬에서는 close(self) 랑 close(self, x) 같은거로 인지함
    # 뒤에 있는 것이 앞에 것을 뒤엎음
    # a = 1
    # a = 2 실행하면  a == 2 인것과 유사함
    def close(self, x):
        self.status = 'closed'

# 추상화는 상속과 관련됨
# 이름이 같은데 다르게 행동함 == polymorphism == 상속과 관련됨
# duck-typing이 뭘까 ?????????????? 

First of all, let's create two instances of the `Door` class and check that the two objects are stored at different addresses

In [6]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')

In [7]:
vars(door1)

{'number': 1, 'status': 'closed'}

In [8]:
vars(door2)

{'number': 1, 'status': 'closed'}

In [9]:
door1 == door2
# 응??????????????? 값이 같은데 왜 다를까
# == operator는 memory 주소비교해서 false 나옴

False

2개의 인스턴스가 인스턴스 객체의 값이 같다고 해서 같은 인스턴스가 아니다!!!!!!!

객체는 주소값이 같아야 같다.

In [79]:
hex(id(door1))

'0xb490a3cc'

In [80]:
hex(id(door2))

'0xb490a3ec'

This confirms that the two instances are separate and unrelated.
The second instance was given the same attributes of the first instance to show that the two are different objects regardless of the value of the attributes.

However if we use `id()` on the class of the two instances we discover that the class is _exactly_ the same

In [81]:
hex(id(door1.__class__))

'0xb497b3ac'

In [82]:
hex(id(door2.__class__))

'0xb497b3ac'

Well this is very important. In Python, a class is not just the schema used to build an object. Rather, the class is a shared living object, which code is accessed at run time.

As we already tested, however, attributes are not stored in the class but in every instance, due to the fact that `__init__()` works on `self` when creating them. Classes, however, can be given attributes like any other object; with a terrific effort of imagination, let's call them _class attributes_.

As you can expect, class attributes are shared among the class instances just like their container

In [83]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status
        
    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'

Pay attention: the `colour` attribute here is not created using `self`, so it is contained in the class and shared among instances

In [84]:
door1 = Door(1, 'closed')
door2 = Door(2, 'closed')

In [85]:
Door.colour
# 클래스 변수는 인스턴스, 클래스 모두 접근 가능

'brown'

In [86]:
door1.colour
# 인스턴스에 없으면 클래스에서 찾는다.

'brown'

In [87]:
door2.colour

'brown'

Until here things are not different from the previous case. Let's see if changes of the shared value reflect on all instances

In [88]:
Door.colour = 'white'
Door.colour

'white'

In [89]:
door1.colour

'white'

In [90]:
door2.colour

'white'

In [91]:
hex(id(Door.colour))

'0xb495c380'

In [92]:
hex(id(door1.colour))

'0xb495c380'

In [93]:
hex(id(door2.colour))

'0xb495c380'

## Raiders of the Lost Attribute

Any Python object is automatically given a `__dict__` attribute, which contains its list of attributes. Let's investigate what this dictionary contains for our example objects:

In [94]:
door1 = Door(1, 'closed')
door2 = Door(2, 'closed')
print(type(Door.__dict__))
Door.__dict__ # class에 dict 쓰면 mappingproxy

<class 'mappingproxy'>


mappingproxy({'colour': 'white', 'open': <function Door.open at 0xb4974f5c>, '__init__': <function Door.__init__ at 0xb4974f14>, '__module__': '__main__', '__doc__': None, '__weakref__': <attribute '__weakref__' of 'Door' objects>, 'close': <function Door.close at 0xb4974fa4>, '__dict__': <attribute '__dict__' of 'Door' objects>})

In [95]:
print(type(door1.__dict__)) # instance는 자기 자신이 가지는 값 나열
door1.__dict__

<class 'dict'>


{'number': 1, 'status': 'closed'}

Leaving aside the difference between a dictionary and a `mappingproxy` object, you can see that the `colour` attribute is listed among the `Door` class attributes, while `status` and `number` are listed for the instance.

How comes that we can call `door1.colour`, if that attribute is not listed for that instance? This is a job performed by the magic `__getattribute__()` method; in Python the dotted syntax automatically invokes this method so when we write `door1.colour`, Python executes `door1.__getattribute__('colour')`. That method performs the  _attribute  lookup_ action, i.e. finds the value of the attribute by looking in different places.

The standard implementation of `__getattribute__()` searches first the internal dictionary (`__dict__`) of an object, then the type of the object itself; in this case `door1.__getattribute__('colour')` executes first `door1.__dict__['colour']` and then, since the latter raises a `KeyError` exception, `door1.__class__.__dict__['colour']`

In [96]:
try:
    door1.__dict__['colour']
except KeyError as e:
    print("Cannot find key {}".format(e))

Cannot find key 'colour'


In [97]:
door1.__class__.__dict__['colour']

'white'

Indeed, if we compare the objects' equality through the `is` operator we can confirm that both `door1.colour` and `Door.colour` are exactly the same object

In [98]:
door1.colour is Door.colour

True

When we try to assign a value to a class attribute directly on an instance, we just put in the `__dict__` of the instance a value with that name, and this value masks the class attribute since it is found first by `__getattribute__()`. As you can see from the examples of the previous section, this is different from changing the value of the attribute on the class itself.

In [99]:
door1.colour = 'white'
door1.__dict__['colour']

'white'

In [100]:
door1.__class__.__dict__['colour']

'white'

In [101]:
Door.colour = 'red'
door1.__dict__['colour']

'white'

In [102]:
door1.__class__.__dict__['colour']

'red'

## Revenge of the Methods

Let's play the same game with methods. First of all you can see that, just like class attributes, methods are listed only in the class `__dict__`. Chances are that they behave the same as attributes when we get them

In [103]:
door1.open is Door.open

False

Whoops. Let us further investigate the matter

In [104]:
Door.__dict__['open']

<function __main__.Door.open>

In [105]:
Door.open

<function __main__.Door.open>

In [106]:
door1.open

<bound method Door.open of <__main__.Door object at 0xb490a2cc>>

So, the class method is listed in the members dictionary as _function_. So far, so good. The same happens when taking it directly from the class; here Python 2 needed to introduce _unbound methods_, which are not present in Python 3. Taking it from the instance returns a _bound method_.

Well, a _function_ is a procedure you named and defined with the `def` statement. When you refer to a function as part of a class in Python 3 you get a plain function, without any difference from a function defined outside a class.

When you get the function from an instance, however, it becomes a bound method. The name _method_ simply means "a function inside an object", according to the usual OOP definitions, while _bound_ signals that the method is linked to that instance. Why does Python bother with methods being bound or not? And how does Python transform a function into a bound method?

First of all, if you try to call a class function you get an error

In [107]:
try:
    Door.open() 
except TypeError as e: # TypeError : type 안맞게 method 사용
    print(e)

open() missing 1 required positional argument: 'self'


Yes. Indeed the function was defined to require an argument called 'self', and calling it without an argument raises an exception. This perhaps means that we can give it one instance of the class and make it work

In [108]:
Door.open(door1)
door1.status

'open'

Python does not complain here, and the method works as expected. So `Door.open(door1)` is the same as `door1.open()`, and this is the difference between a plain function coming from a class an a bound method: the bound method automatically passes the instance as an argument to the function.

Again, under the hood, `__getattribute__()` is working to make everything work and when we call `door1.open()`, Python actually calls `door1.__class__.open(door1)`. However, `door1.__class__.open` is a plain function, so there is something more that converts it into a bound method that Python can safely call.

When you access a member of an object, Python calls `__getattribute__()` to satisfy the request. This magic method, however, conforms to a procedure known as _descriptor protocol_. For the read access `__getattribute__()` checks if the object has a `__get__()` method and calls this latter. So the converstion of a function into a bound method happens through such a mechanism. Let us review it by means of an example.

In [10]:
door1.__class__.__dict__['open'] # method : instance가 쓰는 함수
# 클래스 입장에서 open은 함수라서 함수형태로 보관
# mappingproxy == dict

<function __main__.Door.open(self)>

This syntax retrieves the function defined in the class; the function knows nothing about objects, but it _is_ an object (remember "everything is an object"). So we can look inside it with the `dir()` built-in function

In [110]:
dir(door1.__class__.__dict__['open'])

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [111]:
door1.__class__.__dict__['open'].__get__

<method-wrapper '__get__' of function object at 0xb4974f5c>

As you can see, a `__get__` method is listed among the members of the function, and Python recognizes it as a _method-wrapper_. This method shall connect the `open` function to the `door1` instance, so we can call it passing the instance alone

In [112]:
door1.__class__.__dict__['open'].__get__(door1)

<bound method Door.open of <__main__.Door object at 0xb490a2cc>>

and we get exactly what we were looking for. This complex syntax is what happens behind the scenes when we call a method of an instance.

## When Methods met Classes

Using `type()` on functions defined inside classes reveals some other details on their internal representation

In [113]:
Door.open
# 클래스 입장에서는 open은 함수로 취급한다. == self(instance)따로 집어 넣어야 함
# 인스턴스 입장에서는 method이다. == 첫 인자가 자기 자신이여서 생략해서 사용한다.

<function __main__.Door.open>

In [114]:
door1.open # door1 은 인스턴스라서 method

<bound method Door.open of <__main__.Door object at 0xb490a2cc>>

In [115]:
type(Door.open)

function

In [116]:
type(door1.open)

method

As you can see, Python tells the two apart recognizing the first as a _function_ and the second as a _method_, where the second is a function bound to an instance.

What if we want to define a function that operates on the class instead of operating on the instance? As we may define class attributes, we may also define _class methods_ in Python, through the `classmethod` decorator. Class methods are functions that are bound to the class and not to an instance.

In [117]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod
    def knock(cls):
        print("Knock!")

    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'

Such a definition makes the method callable on both the instance and the class

In [118]:
door1 = Door(1, 'closed')
door1.knock()

Knock!


In [119]:
Door.knock()

Knock!


and Python identifies both as (bound) methods

In [120]:
door1.__class__.__dict__['knock']

In [121]:
door1.knock

<bound method type.knock of <class '__main__.Door'>>

In [122]:
Door.knock

<bound method type.knock of <class '__main__.Door'>>

In [135]:
print(type(Door.knock))

<class 'method'>


In [136]:
print(type(door1.knock))

<class 'method'>


As you can see the `knock()` function accepts one argument, which is called `cls` just to remember that it is not an instance but the class itself. This means that inside the function we can operate on the class, and the class is shared among instances.

# classmethod의 기능
1. 클래스가 function을 method 처럼 사용할 수 있다.
2. 클래스 영역만 바꿀 수 있다.
3. 인스턴스는 인스턴스 변수만 바꿀 수 있었는데, classmethod를 붙여서 class 영역을 바꿀 수 있다.

In [22]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod # class쓸 때 function이 아니라 method 처럼 쓸 수 있다. 첫 번째 인자 안 넣어도 된다. 
    def knock(cls):
        print("Knock!")

    @classmethod # 클래스가 이 함수를 첫 번째 인자를 생략해서 사용할 수 있다.
    def paint(cls, colour):
        cls.colour = colour # 클래스 변수를 바꿈
        
    @classmethod # 클래스 내부의 값을 변경할 때 classmethod를 사용함
    def paint2(cls, number): 
        cls.number = number # 인스턴스 변수를 바꾸는 것이 아니라 클래스 변수를 새로 생성함
    # classmethod decorator 때문에 클래스 변수가 바뀜
        
    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'

In [23]:
# method인지 function인지에 따라서 쓰는 방법이 달라짐
# 클래스가 
x = Door(1, 'open')
Door.open(x) # Door.open() 하려면 instance 넣어야 함
x.open() # method일 때 첫번째 인자 안넣는다.

In [24]:
Door.paint('pink') # class가 function을 method처럼 사용될 수 있다.
Door.colour

'pink'

In [25]:
x.paint('yello') # 클래스 method를 인스턴스가 사용할 수 있다 && 인스턴스가 클래스 변수를 바꿀 수 있다.
print(Door.colour)

yello


In [29]:
x.paint2(6)
print(x.number) # 클래스 변수를 바꾼다!! 인스턴스 변수를 안바꿈

1


The `paint()` classmethod now changes the class attribute `colour` which is shared among instances. Let's check how it works

In [126]:
door1 = Door(1, 'closed')
door2 = Door(2, 'closed')
Door.colour

'brown'

In [127]:
door1.colour

'brown'

In [128]:
door2.colour

'brown'

In [129]:
Door.paint('white')
Door.colour

'white'

In [130]:
door1.colour

'white'

In [131]:
door2.colour

'white'

The class method can be called on the class, but this affects both the class and the instances, since the `colour` attribute of instances is taken at runtime from the shared class.

In [132]:
door1.paint('yellow')
Door.colour

'yellow'

In [133]:
door1.colour

'yellow'

In [134]:
door2.colour

'yellow'

Class methods can be called on instances too, however, and their effect is the same as before. The class method is bound to the class, so it works on this latter regardless of the actual object that calls it (class or instance).

## Movie Trivia

Section titles come from the following movies: _The Empire Strikes Back (1980)_, _Raiders of the Lost Ark (1981)_, _Revenge of the Nerds (1984)_, _When Harry Met Sally (1989)_.

## Sources

You will find a lot of documentation in [this Reddit post](http://www.reddit.com/r/Python/comments/226ahl/some_links_about_python_oop/). Most of the information contained in this series come from those sources.

## Feedback

Feel free to use [the blog Google+ page](https://plus.google.com/u/0/b/110554719587236016835/110554719587236016835/posts) to comment the post. The [GitHub issues](https://github.com/lgiordani/lgiordani.github.com/issues) page is the best place to submit corrections.